$$ ITI \space AI-Pro: \space Intake \space 45 $$
$$ Recommender \space Systems $$
$$ Lab \space no. \space 3 $$

# `01` Import Necessary Libraries

## `i` Default Libraries

In [1]:
import numpy as np
import pandas as pd

## `ii` Additional Libraries
Add imports for additional libraries you used throughout the notebook

In [2]:
from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.accuracy import rmse,mse,mae

----------------------------

# `02` Load Data

Load `songsDataset.csv` file into a dataframe

In [3]:
data = pd.read_csv("Data/songsDataset.csv")
data.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


--------------------------

# `03` Matrix Factorization using Gradient Descent

Practice for Matrix Factorization Implementation

**Matrix Factorization Mathematical Derivation**

We know that the matrix factorization breaks the rating matrix $R$ into two matrices $U \in \mathbb{R}^{\#users \times K}$ and $M \in \mathbb{R}^{K \times \#items}$ where K represent the latent space dimensionality.

$R$ can then be approximated through the following equation:

$$
\mathbf{R} \approx \mathbf{U} \times \mathbf{M} = \hat{\mathbf{R}}
$$

The error, incorporating the regularization term, is calculated as follows:

$$
e_{ij}^2 = (r_{ij} - \sum_{k=1}^K{u_{ik}m_{kj}})^2 + \frac{\beta}{2} \sum_{k=1}^K{(||U||^2 + ||M||^2)}
$$

In order to be able to use Stochastic Gradient Descent (SGD) to optimize $U$ and $M$, we need to find the partial derivatives of the error function with respect to both $u_{ik}$ and $m_{kj}$. The partial derivatives can be derived as follows:

$$
\frac{\partial}{\partial u_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(m_{kj}) + \frac{\beta}{2} \times (2 u_{ik}) = -2 e_{ij} m_{kj} + \beta u_{ik}
$$

$$
\frac{\partial}{\partial m_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(u_{ik}) + \frac{\beta}{2} \times (2 m_{kj}) = -2 e_{ij} u_{ik} + \beta m_{kj}
$$

Thus the update rules will be:

$$
u'_{ik} = u_{ik} + \alpha \frac{\partial}{\partial u_{ik}}e_{ij}^2 = u_{ik} - \alpha(-2 e_{ij} m_{kj} + \beta u_{ik} ) = u_{ik} + \alpha(2 e_{ij} m_{kj} - \beta u_{ik} )
$$

$$
m'_{kj} = m_{kj} + \alpha \frac{\partial}{\partial m_{kj}}e_{ij}^2 = m_{kj} - \alpha(-2 e_{ij} u_{ik} + \beta m_{kj} ) = m_{kj} + \alpha(2 e_{ij} u_{ik} - \beta m_{kj} )
$$

## `0` Construct Utility Matrix from the Data

In [4]:
utility_matrix = data.pivot(index='userID', columns='songID',values='rating')
utility_matrix = utility_matrix.fillna(0)
utility_matrix

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
199980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199988,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `i` Matrices Initialization

Initialize the two random weights matrices $U$ and $M$ (Try $K=3$)

**Note**: Refer to the next cell for the dimensions of $U$ and $M$

**Hine**: You may use a function from `numpy.random` module (see, [Documentation](https://numpy.org/doc/stable/reference/random/index.html))

In [5]:
k = 3
num_users, num_items = utility_matrix.shape
U = np.random.random((num_users, k))
M = np.random.random((k, num_items))

## `ii` Define a Function to Implement Matrix Factorization

**Function Parameters**:
- `R`: Utility Matrix [of shape: (no. of users, no. of items)]
- `U`: User Latent Features Array [of shape: (no. of users, K)]
- `M`: Items Latent Features Array [of shape: (K, no. of items)]
- `epochs`: No. of training epochs
- `lr`: Learning rate (alpha)
- `beta`: Regularization Parameter

**Function Output**:
- `U`: Optimized User Latent Features Array
- `M`: Optimized Items Latent Features Array

**Main Procedures**:
1. Calculate predicted ratings
2. Calculate MSE Error
3. Calculate gradients
4. Update $U$ and $M$


In [6]:
def matrix_factorization(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    """
    Function Parameters:
    - `R`: Utility Matrix [of shape: `(no. of users, no. of items)`]
    - `U`: User Latent Features Array [of shape: `(no. of users, K)`]
    - `M`: Items Latent Features Array [of shape: `(K, no. of items)`]
    - `epochs`: No. of training epochs
    - `lr`: Learning rate (alpha)
    - `beta`: Regularization Parameter

    Function Output:
    - `U`: Optimized User Latent Features Array
    - `M`: Optimized Items Latent Features Array
    """

    U = U.copy()
    M = M.copy()

    # Confirm that no. of features is consistent between U and M
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    # Extract No. of Features (K)
    K = M.shape[0]

    # Define the Epochs loop
    for epoch in range(epochs):
        # Loop over every element in R
        for i in range(U.shape[0]): # Loop over each user
            for j in range(M.shape[1]): # Loop over each item
                if R[i,j] !=0: # Only proceed if the current interaction (i, j) is not missing
                    eij =  R[i,j] - (U[i] @ M.T[j]) # Calculate the error in prediction
                    for d in range(K): # Loop over each latent features dimension
                        # Update Rules for both U and M:
                        U[i, d] = U[i, d] + lr*(2*eij*M[d, j] - beta * U[i, d])
                        M[d, j] = M[d, j] + lr*(2*eij*U[i, d] - beta * M[d, j])

        ## Error Calculation ##
        e_last = e if epoch > 0 else 100000000
        e = 0 # Initialize a variable to accumelate the errors
        for i in range(U.shape[0]): # Loop over each user
            for j in range(M.shape[1]): # Loop over each item
                if R[i,j] !=0: # Only proceed if the current interaction (i, j) is not missing
                                # since we only calculate the error for interactions having a ground truth value

                    first_part =(U[i] @ M.T[j] - R[i,j]) ** 2  # calculate the first part of the error
                    second_part = 0 # Initialize a variable to accumelate the second part of the error

                    for d in range(K): # Loop over each latent features dimension
                        second_part += ((beta / 2) * (U[i,d] ** 2 + M[d,j] ** 2))

                    e += first_part + second_part # accumelate the error to the total error

        print(f'Epoch {epoch+1}/{epochs}: Total Error = {e}')

        if e < 0.001 or e_last-e < 0.001: # Stop if error is so small or improvement is not significant
            break

    return U, M


### `[Bonus]` Vectorized Error Calculation

Can the error calculation part be vectorized to get rid of for loops?

If you would like a challenge, try to redefine the function in the next cell with a vectorized error calculation.

In [7]:
def matrix_factorization_vectorized(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
   
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    R = np.nan_to_num(R, nan=0)
    observed_mask = (R != 0)

    errors = []

    for epoch in range(epochs):
        R_pred = np.dot(U, M)

        E = np.zeros_like(R)
        E[observed_mask] = R[observed_mask] - R_pred[observed_mask]

        U_grad = -2 * np.dot(E, M.T) + beta * U
        M_grad = -2 * np.dot(U.T, E) + beta * M

        U_grad = np.clip(U_grad, -1e2, 1e2)
        M_grad = np.clip(M_grad, -1e2, 1e2)

        U -= lr * U_grad
        M -= lr * M_grad

        first_part = np.sum(E ** 2)
        second_part = beta * (np.sum(U ** 2) + np.sum(M ** 2))
        error = first_part + second_part

        errors.append(error)

        print(f'Epoch {epoch+1}/{epochs}: Total Error = {error}')

        if epoch > 0 and abs(errors[-2] - errors[-1]) < 0.001:
            print("Convergence reached. Stopping early.")
            break

    return U, M

## `iii` Use the Function to to Optimize the $U$ and $V$

In [8]:
U_f, M_f = matrix_factorization(R=utility_matrix.values, U=U, M=M, epochs=1000, lr=0.001, beta=0.01)

Epoch 1/1000: Total Error = 244800.61990947148
Epoch 2/1000: Total Error = 223643.73992362557
Epoch 3/1000: Total Error = 209231.6744301835
Epoch 4/1000: Total Error = 196070.93027832985
Epoch 5/1000: Total Error = 183984.86610619575
Epoch 6/1000: Total Error = 172874.82080098178
Epoch 7/1000: Total Error = 162653.86350963596
Epoch 8/1000: Total Error = 153244.378468854
Epoch 9/1000: Total Error = 144576.34789488878
Epoch 10/1000: Total Error = 136586.230978168
Epoch 11/1000: Total Error = 129216.21671096697
Epoch 12/1000: Total Error = 122413.66667462855
Epoch 13/1000: Total Error = 116130.65299147082
Epoch 14/1000: Total Error = 110323.5482545882
Epoch 15/1000: Total Error = 104952.64933699195
Epoch 16/1000: Total Error = 99981.82826408542
Epoch 17/1000: Total Error = 95378.20795797504
Epoch 18/1000: Total Error = 91111.86223267333
Epoch 19/1000: Total Error = 87155.53968644538
Epoch 20/1000: Total Error = 83484.4109251978
Epoch 21/1000: Total Error = 80075.83822743397
Epoch 22/1000:

In [9]:
U_v, M_v = matrix_factorization_vectorized(R=utility_matrix.values, U=U, M=M, epochs=1000, lr=0.001, beta=0.01)

Epoch 1/1000: Total Error = 709029.469216763
Epoch 2/1000: Total Error = 649680.8320780941
Epoch 3/1000: Total Error = 592645.3337304025
Epoch 4/1000: Total Error = 538019.7311916432
Epoch 5/1000: Total Error = 485978.6130224823
Epoch 6/1000: Total Error = 436759.26392424176
Epoch 7/1000: Total Error = 390643.5700118003
Epoch 8/1000: Total Error = 347937.05484237033
Epoch 9/1000: Total Error = 308945.57708439376
Epoch 10/1000: Total Error = 273950.67352783284
Epoch 11/1000: Total Error = 243170.06207062077
Epoch 12/1000: Total Error = 216732.70242534397
Epoch 13/1000: Total Error = 194637.71534388053
Epoch 14/1000: Total Error = 176523.08977902037
Epoch 15/1000: Total Error = 162110.65895597718
Epoch 16/1000: Total Error = 150838.1839349232
Epoch 17/1000: Total Error = 141766.9448295723
Epoch 18/1000: Total Error = 134274.75024195554
Epoch 19/1000: Total Error = 127759.63958514179
Epoch 20/1000: Total Error = 121766.36544500651
Epoch 21/1000: Total Error = 116185.73357494459
Epoch 22/1

## `iv` Recommend top-K Songs

Recommend top-K ($K=5$) songs for user ($userID=199988$)

Note: Make sure to filter songs they already rated before

In [10]:
def recommend_top_k(user_id, utility_matrix, U, M, k=5):
    user_idx = utility_matrix.index.get_loc(user_id)

    predicted_ratings = np.dot(U[user_idx], M)

    rated_items = utility_matrix.loc[user_id].to_numpy().nonzero()[0]

    predicted_ratings[rated_items] = -np.inf

    top_k_indices = np.argpartition(-predicted_ratings, k)[:k]
    top_k_ratings = predicted_ratings[top_k_indices]

    sorted_indices = np.argsort(-top_k_ratings)
    top_k_indices = top_k_indices[sorted_indices]
    top_k_ratings = top_k_ratings[sorted_indices]

    top_k_song_ids = utility_matrix.columns[top_k_indices]

    print(f"Top-{k} recommendations for User {user_id}:")
    for i, (song_id, rating) in enumerate(zip(top_k_song_ids, top_k_ratings)):
        print(f"\t -Top {i+1} Song: {song_id} (Predicted Rating: {rating:.2f})")

In [11]:
user_id = 199988
recommend_top_k(user_id, utility_matrix, U_v, M_v, k=5)

Top-5 recommendations for User 199988:
	 -Top 1 Song: 45934 (Predicted Rating: 6.09)
	 -Top 2 Song: 119103 (Predicted Rating: 6.01)
	 -Top 3 Song: 60465 (Predicted Rating: 5.87)
	 -Top 4 Song: 132189 (Predicted Rating: 5.71)
	 -Top 5 Song: 74640 (Predicted Rating: 5.69)


------------------------

# `04` Matrix Factorization using SVD Algorithm

Practice for using `SVD` algorithm implementation from `scikit surprise` library.

## `i` Prepare the Data

- Load the Data into `surprise` Dataset
- Split data into train and test


In [12]:
reader = Reader(rating_scale=(1, 5))
dataset = Dataset.load_from_df(data, reader)
trainset, testset = train_test_split(dataset, test_size=0.25)

## `ii` Model Initialization

Instantiate two models:
- Model with baselines (biases)
- Model without baselines

**Note**: Use `surprise.prediction_algorithms.matrix_factorization.SVD` (see, [Documentation](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#:~:text=surprise.prediction_algorithms.matrix_factorization.-,SVD,-(n_factors%3D)))

In [13]:
# Biased Model
model_Biased = SVD()

In [14]:
# Non-Biased Model
model_Non_Biased = SVD(biased=False)

## `iii` Fit each Model on Training Data

In [15]:
# Biased Model
model_Biased.fit(trainset)

In [16]:
# Non-Biased Model
model_Non_Biased.fit(trainset)

## `iv` Test both Models on the Testing Data

Compare the errors of the two models using multiple error formulas.

**Note**: Refer to `surprise.accuracy` module (see, [Documentation](https://surprise.readthedocs.io/en/stable/accuracy.html))

In [17]:
# Biased Model
predictions_model_Biased = model_Biased.test(testset)
rmse(predictions_model_Biased)

RMSE: 1.4968


1.4967914358758765

In [18]:
# Non-Biased Model
predictions_model_Non_Biased = model_Non_Biased.test(testset)
rmse(predictions_model_Non_Biased)

RMSE: 2.0764


2.0764131064902505

## `v` Recommend Top $10$ Songs for User $199988$

Is there a difference in recommended songs from the two models?

In [19]:
# Biased Model
print(f'Top 10 Songs for User 199988 using SVD:')
user_id = 199988
user_index = utility_matrix.index.get_loc(user_id)
predicted_ratings = []
for song_index in range(utility_matrix.shape[1]):
    predicted_ratings.append(model_Biased.predict(user_id, utility_matrix.columns[song_index]).est)
predicted_ratings = np.array(predicted_ratings)
sorted_songs = np.argsort(predicted_ratings)[::-1]
for i in range(10):
    song_id = utility_matrix.columns[sorted_songs[i]]
    print(f'\t{i+1}. SongID = {song_id} With Rating = {predicted_ratings[sorted_songs[i]]:.2f}')


Top 10 Songs for User 199988 using SVD:
	1. SongID = 2726 With Rating = 4.90
	2. SongID = 2263 With Rating = 4.81
	3. SongID = 60465 With Rating = 4.61
	4. SongID = 92881 With Rating = 4.58
	5. SongID = 55240 With Rating = 4.52
	6. SongID = 56660 With Rating = 4.52
	7. SongID = 62954 With Rating = 4.44
	8. SongID = 122065 With Rating = 4.42
	9. SongID = 36561 With Rating = 4.39
	10. SongID = 19299 With Rating = 4.38


In [20]:
# Non-Biased Model
print(f'Top 10 Songs for User 199988 using SVD:')
user_id = 199988
user_index = utility_matrix.index.get_loc(user_id)
predicted_ratings = []
for song_index in range(utility_matrix.shape[1]):
    predicted_ratings.append(model_Non_Biased.predict(user_id, utility_matrix.columns[song_index]).est)
predicted_ratings = np.array(predicted_ratings)
sorted_songs = np.argsort(predicted_ratings)[::-1]
for i in range(10):
    song_id = utility_matrix.columns[sorted_songs[i]]
    print(f'\t{i+1}. SongID = {song_id} With Rating = {predicted_ratings[sorted_songs[i]]:.2f}')


Top 10 Songs for User 199988 using SVD:
	1. SongID = 56660 With Rating = 4.78
	2. SongID = 2726 With Rating = 4.72
	3. SongID = 43267 With Rating = 2.90
	4. SongID = 112023 With Rating = 2.67
	5. SongID = 25182 With Rating = 2.19
	6. SongID = 60888 With Rating = 2.06
	7. SongID = 24427 With Rating = 2.04
	8. SongID = 132189 With Rating = 2.01
	9. SongID = 60465 With Rating = 1.96
	10. SongID = 94604 With Rating = 1.62


----------------------------------------------

$$ Wish \space you \space all \space the \space best \space ♡ $$
$$ Mahmoud \space Shawqi $$